This notebook contains an analysis workflow for refining textures in diffraction spectra using MAUD (http://maud.radiographema.eu/)

## Import packages

In [ ]:
import pathlib

from tqdm.notebook import tqdm
import numpy as np

## Preparing the DAWN spectra

In DAWN the data must be caked with the x-axis chosen as pixel number (pixels) and saved as a .dat file.

It is not possible to adjust data using x-axis as the two-theta angle (degrees).

**Notes on caking in DAWN**

- Import tiffs using either .nxs file (for standard 2 Hz acquisition) or as .dat file (for faster acquisitions) into ‘data slice viewer’ within the ‘processing’ tab.

- Select pilatus, image 2D, finish.

- For **cake remapping** (used in MAUD for texture recalculation and for single peak profile analysis in python), in ‘processing’ window;
    - Import detector calibration (calibration_output.nxs)
    - Threshold mask, set lower to 0.0
    - Cake remapping, azimuthal range 2.5, -357.5, number of azimuthal bins 72, number of bins blank, pixel splitting tick, radial range blank, X axis pixel (MAUD) or angle (SPP)
    - Export to text file, file extension .dat, choose output directory, pad with zeros 5

- For **azimuthal integration** (used in TOPAS for volume fraction), in ‘processing’ window;
    - Import detector calibration (calibration_output.nxs)
    - Threshold mask, set lower to 0.0
    - Azimuthal integration, azimuthal range blank, number of bins blank, pixel splitting tick, radial range blank, X axis angle 
    - Export to text file, file extension .xy, choose output directory, pad with zeros 5
        
*Data can also be caked using pyFAI (https://pyfai.readthedocs.io/en/latest/#)*

## Creating a template ".par" file

Before beginning the batch mode analysis a **template .par file** must be set up in MAUD.
This provides MAUD with information about the beamline setup and also the data to be analysed.

#### Creating and loading beamline calibration

First the beamline calibration data must be loaded into MAUD
- Calibration data for the beamline must be created using a known calibrant. An example of this is shown in the "calibration" folder. The "calibration_1200mm.dat" file is the specrum of a "LaB6" calibrant.
- To be read into MAUD, the calibration data must be converted to a ".esg" file which can be done [using the script below](#cell1)
- Load the .esg calibration file into MAUD.


#### Creating an instrument type
An **'instrument type'** must then be created in MAUD:
- Follow online videos to define the instrument (https://www.youtube.com/user/MaudRietveldProgram/videos)

*Note, the .dat files have undergone calibration in DAWN, so ImageJ isn't needed to load the tiffs, which would lead to a different calibration in MAUD.*

#### Load experimental data into MAUD

- Create a .esg file containing experimental data from the .dat files [using the script below](#cell2).
- Load the data .esg file in MAUD.
- Select the new instrument type defined in the step above.

#### Creating the template .par file

Then, create a **template .par file** in MAUD;

- Set up the refinement;
    - Set data range ~ 2.2 to 9.9
    - Add extra background parameter.
    - Add alpha and beta cifs.
    - Set Biso to 0.5
    - Set approximate volume fraction.
    - Manually adjust crystal paramters to fit peak positions.
    - Auto refine background, free scale factors and cell parameters. *Note, leave beam centre fixed.*
    - Auto refine volume fraction.
    - Auto refine crystal size and micro-strain.
    - Add E-WIMV texture for alpha-phase (set ODF resolution to 5 or 10 degree resolution).
    - Fix crystal size and micro-strain and turn on texture refinement (but, don't refine).
    - *Note, to refine beta-phase, untick ODF refineable on alpha-phase.*
    
- Save this as a template .par file i.e. 'template_065.par'

## Structure of notebook and example file input and output

**1. Write out a single .esg calibration file for MAUD.**
   - *input: caked synchrotron data for calibration.*
        - calibration/calibration_1200mm.dat
   - *output: calibration .esg data file written to 'calibration' folder, named with detector distance.*
        - calibration/calibration_1200mm.esg
        
        
**2. Loop through multiple input files to output .esg data files for MAUD.**
   - *input: caked synchrotron data for experiment.* 
        - data/adc_065_TI64_NDload_900C_15mms_ascii/adc_065_TI64_NDload_900C_15mms_00001.dat
   - *output: experiment .esg data file written to 'calibration' folder.*
        - analysis/065/MAUD_065_00001.esg
        
        
**3. Output many .par analysis files for MAUD batch mode analysis.**
   - *input: caked synchrotron data for experiment.* 
        - data/adc_065_TI64_NDload_900C_15mms_ascii/adc_065_TI64_NDload_900C_15mms_00001.dat
   - *input: template MAUD .par analysis file.* 
        - data/template_par/template_065.par
   - *output: experiment .par analysis files written to 'analysis' folder.*
        - analysis/065/MAUD_065_00001.par
        
        
** 4. Write an instruction file for the batch mode.**
   - *output: instrution file.*
        - MAUD_batch_065_alpha_20iter.ins
        
        
**5. Running MAUD batch mode.**
   - *input: instrution file.*
        - MAUD_batch_065_alpha_20iter.ins
   - *input: experiment .par analysis files.*
        - analysis/MAUD_065_00001.par
   - *output: .par analysis files containing refined alpha texture for each experiment.*
        - analysis/065/MAUD_065_00001_alpha_20iter.par'    
   - *output: text file  containing MAUD refinement results.*
        - analysis/065/batch_results_065_alpha_20iter.txt
        
        
**6. Output many .par analysis files to refine the $\beta$ texture.**
   - *input: .par analysis files containing refined alpha texture for each experiment.*
        - analysis/065/MAUD_065_00001_alpha_20iter.par'
   - *output: .par analysis files for refining beta texture.*
        - analysis/065/MAUD_065_00001_beta.par'
        
   
**7. Write an instruction file for refining the $\beta$ texture in batch mode.**
   - *output: instrution file.*
        - MAUD_batch_065_beta_20iter.ins
        

**8. Running MAUD batch mode (to refine the $\beta$ texture).**
   - *input: instrution file.*
        - MAUD_batch_065_beta_20iter.ins
   - *input: experiment .par analysis files.*
        - analysis/065/MAUD_065_00001_beta.par
   - *output: .par analysis files containing refined beta texture for each experiment.*
        - analysis/065/MAUD_065_00001_beta_20iter.par'    
   - *output: text file  containing MAUD refinement results.*
        - analysis/065/batch_results_065_beta_20iter.txt

<a id="cell1"></a>
## 1. Write out a single .esg calibration file for MAUD

In [ ]:
def dat_to_esg(input_path: str, pixel_size: float, detector_distance: float, start_angle: int, output_path: str):
    """This function reads in a calibration data in '.dat' format and outputs it in '.esg' format
    for reading into MAUD.
    """
    # caked synchrotron data from DAWN
    input_data = np.loadtxt(input_file)

    # read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
    pixel_list = input_data[:, 0] * pixel_size

    cake_spectrum = input_data[:, 1:]
    number_of_cakes = cake_spectrum.shape[1]
    cake_width = 360 / number_of_cakes

    # name and open the output MAUD data file
    output_folder = pathlib.Path(output_path).parent
    output_folder.mkdir(exist_ok=True)

    with open(output_path, 'w') as output_file:

        # write metadata for the top of the file
        output_file.write('_pd_block_id noTitle|#0\n'
        '_diffrn_detector Image Plate\n'
        '_diffrn_detector_type Image Plate\n'
        '_pd_meas_step_count_time ?\n'
        '_diffrn_measurement_method ?\n'
        '_diffrn_measurement_distance_unit mm\n'
        '_pd_instr_dist_spec/detc {:.4f}\n'.format(detector_distance))
        output_file.write('_diffrn_radiation_wavelength ?\n'
        '_diffrn_source_target ?\n'
        '_diffrn_source_power ?\n'
        '_diffrn_source_current ?\n'
        '_pd_meas_angle_omega 0.0\n'
        '_pd_meas_angle_chi 0.0\n'
        '_pd_meas_angle_phi 0.0\n'
        '_riet_par_spec_displac_x 0\n'
        '_riet_par_spec_displac_y 0\n'
        '_riet_par_spec_displac_z 0\n'
        '_riet_meas_datafile_calibrated false\n'
        '_pd_meas_angle_eta {:.1f}\n\n'.format(start_angle))
        output_file.write('loop_\n'
        '_pd_proc_2theta_corrected\n'
        '_pd_meas_intensity_total\n')

        # write the first cake intensity data
        for i in range(len(pixel_list) - 1, -1, -1):
            output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][0]))

        # write all the other cake data with additional info
        for cake_number in range(1, number_of_cakes, 1):    
            output_file.write('\n_pd_block_id noTitle|#{:.0f}\n\n'.format(cake_number))
            cake_angle = start_angle + cake_number * cake_width
            output_file.write('_pd_meas_angle_eta {:.1f}\n\n'.format(cake_angle))
            output_file.write('loop_\n'
            '_pd_proc_2theta_corrected\n'
            '_pd_meas_intensity_total\n')

            # write cake intensity data for each cake
            for i in range(len(pixel_list)-1, -1, -1):
                output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][cake_number]))

    print(f"Written calibration .esg data to file: '{output_path}'.")

In [ ]:
# user inputs (in mm)
detector_distance = 1200 # This is an approximate distance which will be refined by MAUD
pixel_size = 0.296

# specifying the angle of the first cake in the data file.
start_angle = 0

# where the .esg file is written
input_path = "calibration/calibration_1200mm.dat"
output_path = "calibration/calibration_1200mm.esg"

dat_to_esg(input_path, pixel_size, detector_distance, start_angle, output_path)

<a id="cell2"></a>
## 2. Loop through multiple input files to output .esg data files for MAUD

Normally, we have many analysis files spaced in time. These are read in a loop by the `create_data_esgs` which produces one .esg data file for each input .dat file.

In [ ]:
def create_data_esgs(start: int, end: int, step: int, dir_name: str, file_prefix: str, 
                     output_folder: str, pixel_size: float, detector_distance: float, start_angle: int):
    # loop through the data using the image number of the files
    for image_number in tqdm(range(start, end + 1, step)):

        # caked synchrotron data from DAWN
        input_path = dir_name + file_prefix + '_{:05d}.dat'.format(image_number)
        output_path = f"{output_folder}/{image_number:05d}.esg"
        dat_to_esg(input_path, pixel_size, detector_distance, start_angle, output_path)

    print(f"Written {int((end + 1 - start) / step)} .esg data files to '{output_folder}' folder.")

In this case the output files are written into a folder called 'analysis' and saved in the format MAUD_065_00001.esg where 065 is experiment number and 00001 is test number.

In [ ]:
# user inputs (in mm)
detector_distance = 1194.8865 # use the MAUD calibrated detector distance

# test number is put in the name of the input and output file
test_number = 65

# formatting the name of the input files
file_prefix = f'adc_{test_number:03d}_TI64_NDload_900C_15mms'
dir_name = 'example_data/' + file_prefix + '_ascii/'

# number and spacing of files to read
start = 0
end = 10
step = 1

# where the .esg files are written
output_folder = f"analysis/{test_number:03d}"

# Do the file conversion
create_data_esgs(start, end, step, dir_name, file_prefix, output_folder, 
                 pixel_size, detector_distance, start_angle)

## 3. Output many .par analysis files for MAUD batch mode analysis

Write out a number of .par analysis files to be refined using MAUD batch mode. 


Use a template MAUD .par analysis file to swap the data into. This file will have been manually refined in MAUD and setup ready to refine in batch mode. It is assumed that this template MAUD par file is named in the format 'template_065.par', where 065 is the experiment number.


The output files are written into an 'analysis' folder and saved in the format MAUD_065_00001.par where 065 is experiment number and 00001 is test number. This analysis folder will then be used in the subsequent batch mode analysis.

In [ ]:
# user inputs (in mm)
pixel_size = 0.296

# test number is put in the name of the input and output file
test_number = 65

# formatting the name of the input files
file_prefix = 'adc_{:03d}_TI64_NDload_900C_15mms'.format(test_number)
dir_name = 'data/' + file_prefix + '_ascii/'

# number and spacing of files to read
start = 0
end = 10
step = 1

# where the .esg files are written
output_folder = pathlib.Path(f"analysis/{test_number:03d}")

# path to the template_***.par file
par_dir_name = 'data/template_par/'

# name of the data (.esg files) to be replaced in the MAUD template_***.par file
previous_esg_filename = 'MAUD_{:03d}_00000'.format(test_number)

# MAUD file to swap the data into
template_par = par_dir_name + 'template_{:03d}.par'.format(test_number)

In [ ]:
for image_number in tqdm(range(start, end + 1, step)):
    
    # caked synchrotron data from DAWN
    input_file = dir_name + file_prefix + '_{:05d}.dat'.format(image_number)
    input_data = np.loadtxt(input_file)
    # read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
    pixel_list = input_data[:, 0] * pixel_size
    
    cake_spectrum = input_data[:, 1:]
    number_of_cakes = cake_spectrum.shape[1]
    cake_width = 360 / number_of_cakes
   
    # output MAUD analysis file
    output_par_path = output_folder / pathlib.Path(f'MAUD_{test_number:03d}_{image_number:05d}.par')
    
    with open(template_par, 'r') as template_par_file, open(output_par_path, 'w') as new_par_file:
        cake_number = 0
        line = template_par_file.readline() 
            
        while line:
            # rename the datafiles in the .par file
            if previous_esg_filename in line:
                new_esg_filename = 'MAUD_{:03d}_{:05d}'.format(test_number, image_number)
                new_line = line.replace(previous_esg_filename, new_esg_filename)
                new_par_file.write(new_line)
                
            # replace the data when this line is found
            elif '_pd_meas_position _pd_meas_intensity_total _pd_meas_intensity_sigma' in line:
                new_par_file.write(line)
                
                # write the new data in place
                for j in range(len(pixel_list)-1, -1, -1):
                    template_par_file.readline()
                    new_par_file.write('{:.3f}\t{:.8f}\t1.0\n'.format(pixel_list[j], cake_spectrum[j][cake_number]))
                cake_number += 1
            
            # write the rest of the file
            else:
                new_par_file.write(line)
            
            line = template_par_file.readline()
            
print(f"Written {int((end + 1 - start) / step)} .par analysis files to '{output_folder}' folder.")

## 4. Write an instruction file for the batch mode

To run the analysis in batch mode, an instruction file is required to tell MAUD exactly what analysis to do. This section writes the .ins file.

The number 13 corresponds to the refinement analysis in MAUD (texture mode). The number of iterations sets how many times to run the refinement on each sample.

*Note, in multi-phase spectra only one phase can be refined at a time. In dual-phase Ti alloys, the alpha-phase is refined first, followed by the beta. *

In [ ]:
# test number
test_number = 65

# number of refinement iterations
number_of_iterations = 20

# which phase is being refined?
phase = 'alpha'

In [ ]:
# name of instruction file
batch_path = pathlib.Path(f"MAUD_batch_{test_number:03d}_{phase}_{number_of_iterations}iter.ins")

with batch_path.open(mode='w') as output_file:

    output_file.write("loop_\n\
    _riet_analysis_file\n\
    _riet_analysis_iteration_number\n\
    _riet_analysis_wizard_index\n\
    _riet_analysis_fileToSave\n\
    _riet_append_simple_result_to\n\n")
    # note, _riet_meas_datafile_name\n removed from above for swapping the data files.
    # note, _riet_append_result_to\n removed as this only prints out Rwp value.

    for image_number in range (start, end + 1, step):
        output_file.write(f"'/analysis/{test_number:03d}/MAUD_{test_number:03d}_{image_number:05d}.par' "
        f"{number_of_iterations} 13 '/analysis/{test_number:03d}/MAUD_{test_number:03d}_"
        f"{image_number:05d}_{phase}_{number_of_iterations}iter.par' "
        f"'/analysis/{test_number:03d}/batch_results_{test_number:03d}_{phase}_{number_of_iterations}iter.txt\'\n")
print(".ins file written.")

## 5. Running MAUD batch mode

Leave the instruction .ins file and a folder for the analysis (containing the newly created .par analysis files) in a sensible location, such as; 


**/Users/mbcx9cd4/Documents/GitHub/MAUD-batch-analysis/**


_Note, be careful with using a dropbox folder such as /Users/mbcx9cd4/Dropbox\ \(Research\ Group\)/Lightform\ Postdoc/MAUD_


In the terminal navigate to the hidden files within the maud.app using the command; 


**cd /Applications/Maud.app/Contents/Resources/Java**


_Note, you can view these files by right clicking on show package contents._


Then, run the command to start the batch mode;


**java -mx2048M -cp Maud.jar:miscLib.jar:jgaec.jar:ij.jar com.radiographema.MaudText -f /Users/mbcx9cd4/Documents/GitHub/MAUD-batch-analysis/MAUD_batch_065_alpha_20iter.ins**


- -mx2048M sets the memory to be used i.e. 512, 1024, 2048, etc.
- Maud.jar:miscLib.jar:jgaec.jar:ij.jar is a class path to the jar files.
- An absolute path to the instruction file must be given.


The analysis should run automatically. However, sometimes, an *error* appears and the batch mode is cancelled. If this happens, run a command which looks for a file that doesn't exist. A pop-up window will then appear, navigate to the .ins instruction file and click open. 


The batch mode will then run, showing the refinements in the terminal and outputting the refined analysis files and a result text file.

## 6. Output many .par analysis files to refine the $\beta$ texture

In [ ]:
# test number is put in the name of the input and output file
test_number = 65

# previous information for the input file
number_of_iterations = 20
phase1 = 'alpha'

# number and spacing of files to read
start = 0
end = 10
step = 1

# where the .esg files are written
output_folder = pathlib.Path(f"analysis/{test_number:03d}")

# which phase is being refined?
phase2 = 'beta'

In [ ]:
for image_number in tqdm(range(start, end + 1, step)):

    second_iteration = False
    
    # MAUD analysis file with refined alpha texture
    input_par_path = f'analysis/{test_number:03d}/MAUD_{test_number:03d}_{image_number:05d}_{phase1}_{number_of_iterations:02d}iter.par'
   
    # output MAUD analysis file for refining beta texture
    output_par_path = output_folder / pathlib.Path(f'MAUD_{test_number:03d}_{image_number:05d}_{phase2}.par')
    
    with open(input_par_path, 'r') as template_par_file, open(output_par_path, 'w') as new_par_file:
        line = template_par_file.readline() 
            
        while line:
            # remove the max reflection plane indices from the previous refinement
            if '_refln_index_h _refln_index_k _refln_index_l _refln_F_squared_meas _refln_F_squared_calc \
_refln_F_squared_sigma' in line and second_iteration == False:
                new_par_file.write(line.strip('\n'))
                for i in range(0,22):
                    line = template_par_file.readline()
                second_iteration = True
             
            # remove the max reflection plane indices from the previous refinement
            elif '_refln_index_h _refln_index_k _refln_index_l _refln_F_squared_meas _refln_F_squared_calc \
_refln_F_squared_sigma' in line and second_iteration == True:
                new_par_file.write(line.strip('\n'))
                for i in range(0,7):
                    line = template_par_file.readline()
                second_iteration = False
                
            # set refinement of alpha ODF to false
            elif '_rita_odf_refinable true' in line:
                new_par_file.write('_rita_odf_refinable false\n')
                
            elif '#subordinateObject_none tex' in line:
                for l in range (0, 7):
                    line = template_par_file.readline()
                # write metadata for the top of the file
                new_par_file.write('#subordinateObject_E-WIMV\n\n'
                '_pd_proc_ls_pref_orient_corr \'E-WIMV\'\n\n'
                '_rita_generate_symmetry none\n'
                '_rita_wimv_sum_coincidence true\n'
                '_rita_wimv_iteration_max 10\n'
                '_rita_wimv_exponent 0.01\n'
                '_rita_wimv_refl_min_int 0.001\n'
                '_rita_wimv_odf_resolution 10\n'
                '_rita_wimv_tube_projection true\n'
                '_rita_wimv_store_ang_conv true\n'
                '_rita_wimv_odf_coverage_% 0\n'
                '_rita_odf_sharpness ?\n'
                '_rita_wimv_phon_use ?\n'
                '_rita_wimv_tube_weight 0.5\n'
                '_rita_wimv_normalize_pole_figures true\n'
                '_rita_wimv_weigths_exponent 0.5\n'
                '_rita_odf_refinable true\n'
                '_rita_wimv_refl_min_dspacing 0.0\n\n'
                '#custom_object_odf\n'
                'loop_\n'
                '_rita_wimv_odf_values\n')

                for i in range(0, 10):
                    for j in range(0, 10):
                        for k in range (0, 37):
                            new_par_file.write('1.0 ')
                        new_par_file.write('\n')  
                    new_par_file.write('\n')
                new_par_file.write('\n')
                
                new_par_file.write('#end_custom_object_odf\n\n\n'
                '#end_subordinateObject_E-WIMV\n\n\n')
            
            # write the rest of the file
            else:
                new_par_file.write(line)
            
            line = template_par_file.readline()
            
print(f"Written {int((end + 1 - start) / step)} .par analysis files to '{output_folder}' folder.")

## 7. Write an instruction file for refining the $\beta$ texture in batch mode

In [ ]:
# user inputs

# test number
test_number = 65

# number of refinement iterations
number_of_iterations = 20

# which phase is being refined?
phase = 'beta'

In [ ]:
# name of instruction file
batch_path = pathlib.Path(f"MAUD_batch_{test_number:03d}_{phase}_{number_of_iterations}iter.ins")

with batch_path.open(mode='w') as output_file:

    output_file.write("loop_\n\
    _riet_analysis_file\n\
    _riet_analysis_iteration_number\n\
    _riet_analysis_wizard_index\n\
    _riet_analysis_fileToSave\n\
    _riet_append_simple_result_to\n\n")
    # note, _riet_meas_datafile_name\n removed from above for swapping the data files.
    # note, _riet_append_result_to\n removed as this only prints out Rwp value.

    for image_number in range (start, end + 1, step):
        output_file.write(f"'/analysis/{test_number:03d}/MAUD_{test_number:03d}_{image_number:05d}_{phase}.par' "
        f"{number_of_iterations} 13 '/analysis/{test_number:03d}/MAUD_{test_number:03d}_"
        f"{image_number:05d}_{phase}_{number_of_iterations}iter.par' "
        f"'/analysis/{test_number:03d}/batch_results_{test_number:03d}_{phase}_{number_of_iterations}iter.txt\'\n")
print(".ins file written.")